In [1]:
import torch
import e3nn
import json
import cuequivariance as cue
from fused_e3nn import fused_uvu_TP

Using /home2/lsy/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home2/lsy/.cache/torch_extensions/py311_cu124/sptp_linear/build.ninja...
Building extension module sptp_linear...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module sptp_linear...


In [2]:
def mul_Irreps(mul, i_in):
    dd = []
    for ori_mul, ir in i_in:
        dd.append((ori_mul*mul, (ir.l, ir.p)))
    return e3nn.o3.Irreps(dd)
def compare(a, b):
    isclose = torch.isclose(a, b)
    diff_pos = torch.argwhere(isclose == False)
    anything_bad = False
    for pos in diff_pos:
        pos_t = [x for x in pos]
        if(abs(a[pos_t] - b[pos_t]) > 1e-6):
            anything_bad = True
            print(pos)
            print(a[pos_t] - b[pos_t] )
    if(not anything_bad):
        print("All Good")
            
IR_IN1_IDX = 0
IR_IN2_IDX = 1
IR_OUT_IDX = 2
INST_IDX = 3

def load_nequip_config(h, l_max, layer_idx):
    filename = f"/home2/lsy/mdsim/nequip/benchmark_config/4_{h}_{l_max}_p_sc.txt"
    with open(filename, "r") as f:
        f_in = f.read().split("\n")

    per_layer_dict = dict()
    for l_idx, d in enumerate(f_in):
        if(d == "") : continue
        dd = json.loads(d)
        per_layer_dict[l_idx] = dd
    tp_list = per_layer_dict[layer_idx]["tp"]
    i_in1 = e3nn.o3.Irreps(tp_list[IR_IN1_IDX])
    i_in2 = e3nn.o3.Irreps(tp_list[IR_IN2_IDX])
    i_out = e3nn.o3.Irreps(tp_list[IR_OUT_IDX])
    inst_tuple = [tuple(x) for x in tp_list[INST_IDX]]

    return i_in1, i_in2, i_out, inst_tuple


In [3]:
def load_nequip_config_e3nn_cueq(h, l_max, layer_idx):
    filename = f"/home2/lsy/mdsim/nequip/benchmark_config/4_{h}_{l_max}_p_sc.txt"
    with open(filename, "r") as f:
        f_in = f.read().split("\n")

    per_layer_dict = dict()
    for l_idx, d in enumerate(f_in):
        if(d == "") : continue
        dd = json.loads(d)
        per_layer_dict[l_idx] = dd
    tp_list = per_layer_dict[layer_idx]["tp"]

    ei_in1 = e3nn.o3.Irreps(tp_list[IR_IN1_IDX])
    ei_in2 = e3nn.o3.Irreps(tp_list[IR_IN2_IDX])
    ei_out = e3nn.o3.Irreps(tp_list[IR_OUT_IDX])
    inst_tuple = [tuple(x) for x in tp_list[INST_IDX]]


    # changing mul for each ir.l
    new_in1_list = []
    new_out_list = []
    changed_idx = [[],[]]
    # mul_list = {}
    mul_list = {0:128, 1:64}

    for idx, (mul,ir) in enumerate(ei_in1):
        if (ir.l in mul_list):
            new_in1_list.append((mul_list[ir.l], ir))
            for inst in inst_tuple:
                if(idx == inst[0]):
                    changed_idx[0].append(inst[2])
                    changed_idx[1].append(mul_list[ir.l])
        else:
            new_in1_list.append((mul, ir))

    for idx, (mul,ir) in enumerate(ei_out):
        if (idx in changed_idx[0]):
            new_out_list.append((changed_idx[1][changed_idx[0].index(idx)], ir))
        else:
            new_out_list.append((mul, ir))

    ei_in1 = e3nn.o3.Irreps(new_in1_list)
    ei_out = e3nn.o3.Irreps(new_out_list)

    ci_in1 = cue.Irreps("O3", str(ei_in1))
    ci_in2 = cue.Irreps("O3", tp_list[IR_IN2_IDX])
    ci_out = cue.Irreps("O3", str(ei_out))


    return [ei_in1,ei_in2,ei_out,inst_tuple] , [ci_in1,ci_in2,ci_out,inst_tuple]


In [4]:
def to_cuda_list(*arg, input_dtype = torch.float32):
    return_list = []
    for item in arg:
        if(type(item) == torch.Tensor):
            return_list.append(item.to(device="cuda"))
        else:
            return_list.append(torch.tensor(item,device="cuda", dtype=input_dtype))
    return return_list

def to_cuda_dict(strname_list, *arg):
    return_dict = {}
    for item,name in zip(arg,strname_list):
        if(type(item) == torch.Tensor):
            return_dict[name] = item.to("cuda")
        else:
            return_dict[name] = torch.tensor(item,device="cuda")
    return return_dict

def cumsum_list(s, np1 = True):
    new_s = []
    current = 0
    for e in s:
        new_s.append(current)
        current += e
    if(np1):
        new_s.append(current)
    return new_s

In [5]:
h = 32
l_max = 2
layer_idx = 3
batch_size = 4096
# i_in1, i_in2, i_out, inst_tuple = load_nequip_config(h,l_max,layer_idx)
# i_in2 = mul_Irreps(3, i_in2)
e3nn_config, cueq_config = load_nequip_config_e3nn_cueq(h,l_max,layer_idx)
i_in1, i_in2, i_out, inst_tuple = e3nn_config

# not really needed for v=1 
uvuv_tp = e3nn.o3.FullTensorProduct(i_in1,i_in2, filter_ir_out=i_out)
uvuv_i_out = uvuv_tp.irreps_out

# split_size = []
# reshape_size = []
# for inst in uvuv_tp.instructions:
#     split_size.append(uvuv_i_out[inst.i_out].dim)
#     reshape_size.append([inst.path_shape[0],inst.path_shape[1],uvuv_i_out[inst.i_out][1].dim])
# weight_mul = e3nn.o3.experimental.FullTensorProduct_uvu_weight_only(i_in1, i_in2, split_size, reshape_size, filter_ir_out=i_out, irrep_normalization=None, regroup_output=False)
# uvw
# i_out = e3nn.o3.Irreps(tp_list[IR_OUT_IDX])
# tp = e3nn.o3.FullyConnectedTensorProduct(i_in1,i_in2,i_out,shared_weights=False, internal_weights=False)

# # uvu
tp = e3nn.o3.TensorProduct(i_in1,i_in2,i_out,inst_tuple,shared_weights=False, internal_weights=False)

fused_tp = fused_uvu_TP(l_max, i_in1,i_in2,i_out,inst_tuple,per_block_batch=4, device="cuda")

/home2/lsy/miniconda3/envs/cueq/lib/python3.11/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home2/lsy/miniconda3/envs/cueq/lib/python3.11/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home2/lsy/miniconda3/envs/cueq/lib/python3.11/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/hom

In [6]:
torch.manual_seed(0)

in1 = torch.rand(batch_size, i_in1.dim, device="cuda", requires_grad=True)
in2 = torch.rand(batch_size, i_in2.dim, device="cuda", requires_grad=True)
# weight = torch.ones(batch_size,tp.weight_numel, device="cuda", requires_grad=True)
weight = torch.rand(batch_size,tp.weight_numel, device="cuda", requires_grad=True)

in1_c = in1.detach().clone()
in2_c = in2.detach().clone()
weight_c = weight.detach().clone()
in1_c.requires_grad =True
in2_c.requires_grad =True
weight_c.requires_grad =True

In [7]:
tp = tp.cuda()

In [8]:
out_exp = tp(in1,in2,weight)
y = torch.nn.functional.gelu(out_exp).sum()
f_in1, f_in2, f_weight = torch.autograd.grad(y, [in1,in2,weight], create_graph=True)
# out_exp.grad = None

f_in1_gelu = torch.nn.functional.gelu(f_in1)
f_in2_gelu = torch.nn.functional.gelu(f_in2)
f_weight_gelu = torch.nn.functional.gelu(f_weight)
fake_loss = f_in1_gelu.sum() + f_in2_gelu.sum() + f_weight_gelu.sum()
fake_loss.backward()

In [9]:
out_ours = fused_tp(in1_c,in2_c,weight_c)
y_ours = torch.nn.functional.gelu(out_ours).sum()
f_in1_c, f_in2_c, f_weight_c = torch.autograd.grad(y_ours, [in1_c,in2_c,weight_c], create_graph=True)
# out_ours.grad = None

f_in1_gelu_c = torch.nn.functional.gelu(f_in1_c)
f_in2_gelu_c = torch.nn.functional.gelu(f_in2_c)
f_weight_gelu_c = torch.nn.functional.gelu(f_weight_c)
fake_loss_c = f_in1_gelu_c.sum() + f_in2_gelu_c.sum() + f_weight_gelu_c.sum()
fake_loss_c.backward()

In [14]:
weight.grad

tensor([[0.5754, 0.5542, 0.2607,  ..., 3.4840, 4.6964, 3.2508],
        [1.3899, 0.8044, 0.6463,  ..., 0.3992, 0.6009, 1.1369],
        [1.8709, 1.4929, 0.9988,  ..., 3.0580, 2.0395, 1.0270],
        ...,
        [1.1788, 1.4602, 1.9032,  ..., 4.9027, 1.3650, 2.8675],
        [0.6461, 0.3524, 1.3731,  ..., 0.3250, 2.1968, 1.5510],
        [0.4353, 0.7938, 0.8230,  ..., 2.5534, 1.3907, 3.4741]],
       device='cuda:0')

In [15]:
weight_c.grad

tensor([[0.5754, 0.5542, 0.2607,  ..., 3.4840, 4.6964, 3.2508],
        [1.3899, 0.8044, 0.6463,  ..., 0.3992, 0.6009, 1.1369],
        [1.8709, 1.4929, 0.9988,  ..., 3.0580, 2.0395, 1.0270],
        ...,
        [1.1788, 1.4602, 1.9032,  ..., 4.9027, 1.3650, 2.8675],
        [0.6461, 0.3524, 1.3731,  ..., 0.3250, 2.1968, 1.5510],
        [0.4353, 0.7938, 0.8230,  ..., 2.5534, 1.3907, 3.4741]],
       device='cuda:0')